In [75]:
import pandas as pd
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM, GlobalAvgPool1D
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('/content/Reviews.csv',on_bad_lines="skip")

In [4]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
df["sentiment"]=df['Score'].apply(lambda x:1 if x>=4 else 0)

In [6]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,1


In [8]:
def clean_text(text):
  text=str(text).lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text) # Убирает спиц сиволы
  text = re.sub(r'\s+', ' ', text).strip() #Убирает пробелы
  return text

In [9]:
df["Text"]=df['Text'].apply(clean_text)

In [11]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,i have bought several of the vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled as jumbo salted peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,if you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffy at a great price there was a wide ...,1


In [15]:
df["Text"][5]

'i got a wild hair for taffy and ordered this five pound bag the taffy was all very enjoyable with many flavors watermelon root beer melon peppermint grape etc my only complaint is there was a bit too much redblack licoriceflavored pieces just not my particular favorites between me my kids and my husband this lasted only two weeks i would recommend this brand of taffy it was a delightful treat'

In [16]:
vocab=100000
max_length=150
trunc_type="post"
padding_type="post"
oov_token='<OOV>'

In [20]:
tokenizer=Tokenizer(num_words=vocab,oov_token=oov_token)
tokenizer.fit_on_texts(df["Text"])

In [21]:
X=df["Text"]
Y=df["sentiment"]

X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.1,random_state=42)

In [33]:
train_token=tokenizer.texts_to_sequences(X_train)
test_token=tokenizer.texts_to_sequences(X_test)

In [34]:
train_token[9]

[3,
 186,
 2,
 3661,
 178,
 8,
 614,
 14,
 21,
 945,
 6,
 305,
 16,
 3661,
 1136,
 649,
 3,
 1015,
 67,
 2165,
 5,
 178,
 16,
 3661,
 1136,
 649,
 4,
 321,
 649,
 3,
 1776,
 2,
 44,
 73,
 3,
 145,
 18,
 1322,
 91,
 3,
 186,
 85,
 432,
 1581,
 13,
 142,
 73,
 22,
 432,
 3419,
 3,
 51,
 60,
 2,
 176,
 44,
 39,
 186,
 115]

In [35]:
train_token=pad_sequences(train_token,maxlen=max_length,padding=padding_type,truncating=trunc_type)
test_token=pad_sequences(test_token,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [39]:
len(train_token[163])

150

In [62]:
embeding_dim=100
model=Sequential(
    [
        Embedding(vocab,embeding_dim,input_length=max_length),
        GlobalAvgPool1D(),
        Dense(64, activation="relu"),
        Dropout(0.5),
        Dense(32,  activation="relu"),
        Dropout(0.3),
        Dense(1,activation="sigmoid")
    ]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [63]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy",Precision(),Recall()]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [66]:
early_stopping=EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=3,
    restore_best_weights=True
)

In [60]:
len(train_token)

32674

In [79]:
history=model.fit(
    train_token,Y_train,
    epochs=20,
    batch_size=32,
    validation_data=(test_token, Y_test),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/20
1022/1022 ━━━━━━━━━━━━━━━━━━━━ 148s 144ms/step - accuracy: 0.8971 - loss: 0.2615 - precision_2: 0.9200 - recall_2: 0.9479 - val_accuracy: 0.8469 - val_loss: 0.3541 - val_precision_2: 0.9324 - val_recall_2: 0.8647
Epoch 2/20
 787/1022 ━━━━━━━━━━━━━━━━━━━━ 34s 145ms/step - accuracy: 0.9154 - loss: 0.2268 - precision_2: 0.9337 - recall_2: 0.9584

KeyboardInterrupt: 

In [67]:
loss, acuracy, precision, recall=model.evaluate(test_token,Y_test)

114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8614 - loss: 0.3208 - precision_2: 0.8828 - recall_2: 0.9445


In [68]:
loss

0.31394603848457336

In [69]:
acuracy

0.8705590963363647

In [70]:
precision

0.8894070386886597

In [71]:
recall

0.9508547186851501

In [77]:
y_pred=model.predict(test_token)>0.5

114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [78]:
print(classification_report(Y_test,y_pred,target_names=["Negative","Positive"]))

              precision    recall  f1-score   support

    Negative       0.78      0.60      0.68       823
    Positive       0.89      0.95      0.92      2808

    accuracy                           0.87      3631
   macro avg       0.84      0.77      0.80      3631
weighted avg       0.86      0.87      0.86      3631

